In [4]:
import os
import pandas as pd
import numpy as np

import tensorflow as tf
import tensorflow_data_validation as tfdv
import tensorflow_transform as tft
import tfx

from tfx.v1.orchestration import LocalDagRunner
from tfx.orchestration.experimental.interactive.interactive_context import \
    InteractiveContext
from tfx.components import (ExampleValidator, ImportExampleGen, CsvExampleGen,
                            SchemaGen, StatisticsGen, Trainer, Transform)
from tfx.proto import example_gen_pb2

from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

from tf_pipeline import create_pipeline

# %matplotlib inline
%config InlineBackend.figure_format = 'retina'
%load_ext autoreload
%autoreload

In [16]:
data = pd.read_csv('./data/train.csv')
train, val = train_test_split(data, random_state=42, test_size=0.2)
train.to_csv('./data/data_for_tfx/train.csv', index=False)
val.to_csv('./data/data_for_tfx/val.csv', index=False)

In [71]:
for c in range(53):
    print(c, data[[str(c), 'target']].corr().values[0, 1])

0 0.0024274479506105964
1 -0.005544974540257774
2 0.0010031247928262241
3 0.0028069322892188267
4 -0.0013002414169083214
5 -0.0021248807869583067
6 0.000666187081653167
7 0.012102971847687905
8 0.00034659860396916775
9 0.0015044400252109838
10 0.003395601215273004
11 -0.001565620379087832
12 0.0020162461054601733
13 -0.0010088678148505903
14 -0.0005130414287193492
15 0.0016529107380790866
16 -0.00085300570784962
17 0.0029526560177733693
18 0.0011971305328859188
19 0.0015196647859598636
20 -0.0004558658671888102
21 0.002873691811380274
22 0.00030376701109259346
23 0.0012801161331549155
24 0.0010702037435782684
25 0.004276293074090819
26 0.0063696901317545586
27 -0.0014589039805130114
28 0.00345053169263704
29 -0.004785289112689035
30 -0.0009638460682863181
31 0.005076866488022204
32 0.0005612219703133269
33 -0.00230129628691916
34 0.0013313336886157695
35 -0.006267346939762631
36 0.002623667339345591
37 0.0016419044702203815
38 -0.0038445239475503484
39 0.006283336444491364
40 -0.007215

# TFX Pipeline

In [17]:
name = 'eda_pipe'
data_root = "./data_for_tfx/"
context = InteractiveContext()

## Load dataset

In [18]:
input_cnf = example_gen_pb2.Input(splits=[
                    example_gen_pb2.Input.Split(name='train', 
                                                pattern=f'train*'),
                    example_gen_pb2.Input.Split(name='eval', pattern=f'val*'),
                ])

example_gen = CsvExampleGen(input_base=data_root, input_config=input_cnf)

context.run(example_gen)

ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 1
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=CsvExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

In [19]:
## Data validation

In [20]:
statistics_gen = StatisticsGen(
    examples=example_gen.outputs['examples'])
context.run(statistics_gen)

context.show(statistics_gen.outputs['statistics'])

In [22]:
schema_gen = SchemaGen(
    statistics=statistics_gen.outputs['statistics'],
    infer_feature_shape=True)
context.run(schema_gen)

context.show(schema_gen.outputs['schema'])

,Type,Presence,Valency,Domain
Feature name,,,,
'0',INT,required,,-
'1',INT,required,,-
'10',INT,required,,-
'11',INT,required,,-
'12',INT,required,,-
'13',FLOAT,required,,-
'14',FLOAT,required,,-
'15',FLOAT,required,,-
'16',FLOAT,required,,-


In [23]:
example_validator = ExampleValidator(
    statistics=statistics_gen.outputs['statistics'],
    schema=schema_gen.outputs['schema'])
context.run(example_validator)

context.show(example_validator.outputs['anomalies'])

## Transform dev cells

According to EDA we have such columns and types: 

- uniform distributed columns withing 0 - 499 range: 0 - 5, 9 - 12 
- reversed pyramid: 6 
- uniform distributed columns withing 0 - 1 range: 7
- categorical columns {0, 1}: 8 
- uniform distributed columns withing 0 - 14 range: 13 - 52

We can take small amount to build fast working interactive TFX cmponent.
But due to the fact that we have small amount of data we can use them for debuggin our transform component of future pipeline. 
After schema and transformed cached dataset is good enough we can integrate all out preprocessing pipeline components and get transformed data and preprocessing artefacts. 

In [75]:
%%writefile transform.py

import tensorflow as tf
import tensorflow_transform as tft

def preprocessing_fn(inputs):
    
    schema = {'big_columns' : [str(x) for x in list(range(6))] + [str(x) for x in list(range(9, 13))], 
              'medium_columns' : [str(x) for x in list(range(13, 53))], 
              'small_columns': ['7'],
              }
    outs = {}
    
    for column_type in ['big_columns', 'medium_columns', 'small_columns']:
        columns = []
        for c in schema[column_type]:
            columns.append(inputs[c])                          
        outs[column_type] = tft.scale_by_min_max(tf.concat(columns, axis=-1),
                                                 output_min=-0.5,
                                                 output_max=0.5,
                                                 elementwise=True)
    
    outs['categorical'] = tf.cast(inputs['8'], tf.int64)
    outs['z_normal'] = tft.scale_to_z_score(10 - inputs['6'])
    outs['target'] = inputs['target']
        
    return outs

Overwriting transform.py


In [59]:
transform = Transform(
    examples=example_gen.outputs['examples'],
    schema=schema_gen.outputs['schema'],
    module_file=os.path.abspath("transform.py"))
context.run(transform)

running bdist_wheel
running build
running build_py
creating build
creating build/lib
copying tf_pipeline.py -> build/lib
copying transform.py -> build/lib
installing to /tmp/tmp_a2moy7m
running install
running install_lib
copying build/lib/transform.py -> /tmp/tmp_a2moy7m
copying build/lib/tf_pipeline.py -> /tmp/tmp_a2moy7m
running install_egg_info


/home/babych/miniconda3/envs/tfx_14/lib/python3.10/site-packages/setuptools/_distutils/cmd.py:66: SetuptoolsDeprecationWarning: setup.py install is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` directly.
        Instead, use pypa/build, pypa/installer or other
        standards-based tools.

        See https://blog.ganssle.io/articles/2021/10/setup-py-deprecated.html for details.
        ********************************************************************************

!!
  self.initialize_options()


running egg_info
creating tfx_user_code_Transform.egg-info
writing tfx_user_code_Transform.egg-info/PKG-INFO
writing dependency_links to tfx_user_code_Transform.egg-info/dependency_links.txt
writing top-level names to tfx_user_code_Transform.egg-info/top_level.txt
writing manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
reading manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
writing manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
Copying tfx_user_code_Transform.egg-info to /tmp/tmp_a2moy7m/tfx_user_code_Transform-0.0+657fa7624a3641fc4b4fb7a1f307f23c77a7f82093b8fbd7419acdbcf99d7512-py3.10.egg-info
running install_scripts
creating /tmp/tmp_a2moy7m/tfx_user_code_Transform-0.0+657fa7624a3641fc4b4fb7a1f307f23c77a7f82093b8fbd7419acdbcf99d7512.dist-info/WHEEL
creating '/tmp/tmp_8ocagfv/tfx_user_code_Transform-0.0+657fa7624a3641fc4b4fb7a1f307f23c77a7f82093b8fbd7419acdbcf99d7512-py3-none-any.whl' and adding '/tmp/tmp_a2moy7m' to it
adding 'tf_pipeline.py'
a


[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: pip install --upgrade pip


Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Processing /tmp/tfx-interactive-2024-07-17T09_17_56.510132-zkg7gy7h/_wheels/tfx_user_code_Transform-0.0+657fa7624a3641fc4b4fb7a1f307f23c77a7f82093b8fbd7419acdbcf99d7512-py3-none-any.whl


...

INFO:tensorflow:struct2tensor is not available.


INFO:tensorflow:struct2tensor is not available.


INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_decision_forests is not available.


ExecutionResult(
    component_id: Transform
    execution_id: 10
    outputs:
        transform_graph: OutputChannel(artifact_type=TransformGraph, producer_component_id=Transform, output_key=transform_graph, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        transformed_examples: OutputChannel(artifact_type=Examples, producer_component_id=Transform, output_key=transformed_examples, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        updated_analyzer_cache: OutputChannel(artifact_type=TransformCache, producer_component_id=Transform, output_key=updated_analyzer_cache, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        pre_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=pre_transform_schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        pre_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=pre_transform_stats, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        post_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=post_transform_schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        post_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=post_transform_stats, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        post_transform_anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=Transform, output_key=post_transform_anomalies, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

In [60]:
transfromed_uri = transform.outputs['transformed_examples'].get()[0].uri
post_transform_schema_uri = transform.outputs['post_transform_schema'].get()[0].uri
print(transfromed_uri, post_transform_schema_uri)

/tmp/tfx-interactive-2024-07-17T09_17_56.510132-zkg7gy7h/Transform/transformed_examples/10 /tmp/tfx-interactive-2024-07-17T09_17_56.510132-zkg7gy7h/Transform/post_transform_schema/10


In [61]:
tr_schema = tfdv.load_schema_text(post_transform_schema_uri + "/schema.pbtxt")
feature_spec = tft.tf_metadata.schema_utils.schema_as_feature_spec(tr_schema).feature_spec
feature_spec

{'big_columns': FixedLenFeature(shape=[10], dtype=tf.float32, default_value=None),
 'categorical': FixedLenFeature(shape=[1], dtype=tf.int64, default_value=None),
 'medium_columns': FixedLenFeature(shape=[40], dtype=tf.float32, default_value=None),
 'small_columns': FixedLenFeature(shape=[1], dtype=tf.float32, default_value=None),
 'target': FixedLenFeature(shape=[1], dtype=tf.float32, default_value=None),
 'z_normal': FixedLenFeature(shape=[1], dtype=tf.float32, default_value=None)}

In [62]:
tfdv.display_schema(tr_schema)

,Type,Presence,Valency,Domain
Feature name,,,,
'big_columns',FLOAT,required,,-
'categorical',INT,required,,-
'medium_columns',FLOAT,required,,-
'small_columns',FLOAT,required,,-
'target',FLOAT,required,,-
'z_normal',FLOAT,required,,-


In [63]:
! ls -l {transfromed_uri}/Split-eval

total 3564
-rw-rw-r-- 1 babych babych 3646032 Jul 17 11:24 transformed_examples-00000-of-00001.gz


In [64]:
val_file_pattern = os.path.join(transfromed_uri, 'Split-eval/transformed_examples-00000-of-00001.gz')

AUTOTUNE = tf.data.AUTOTUNE
BATCH = 10

val_ds = tf.data.experimental.make_batched_features_dataset(
    file_pattern=val_file_pattern,
    batch_size=BATCH,
    features=feature_spec,
    reader=tf.data.TFRecordDataset,
    reader_args=['GZIP'],
    shuffle=False)

val_ds = val_ds.prefetch(buffer_size=AUTOTUNE)

In [65]:
for x in val_ds:
    break
x

{'big_columns': <tf.Tensor: shape=(10, 10), dtype=float32, numpy=
 array([[-0.22945893,  0.05711424,  0.28757513,  0.31162328,  0.3216433 ,
         -0.11523047,  0.05511022, -0.45991984,  0.21342683,  0.17935872],
        [ 0.03106213, -0.26152307,  0.04108214,  0.22545087,  0.4719439 ,
          0.3416834 ,  0.3557114 ,  0.38977957,  0.19739479, -0.36573148],
        [-0.02304611, -0.10521042,  0.3957916 , -0.19138277,  0.13527054,
         -0.249499  , -0.18136272,  0.44388777,  0.26152307, -0.09318638],
        [-0.35771543, -0.43386775, -0.00901803,  0.28757513, -0.45791584,
          0.09118235, -0.48997995, -0.251503  , -0.46793586,  0.3957916 ],
        [ 0.28557116, -0.31563127,  0.36372745,  0.00901806,  0.04308617,
         -0.2715431 , -0.0250501 ,  0.4218437 , -0.28156313,  0.38577157],
        [ 0.21943885, -0.46793586, -0.43587175,  0.23146296,  0.21943885,
          0.3056112 , -0.38176352, -0.47194389, -0.3256513 ,  0.0250501 ],
        [ 0.09519041, -0.08717436,  0.09

## Prepare dataset

After we have checked the output of the data we can run transform pipeline on the whole dataset. 

In [74]:
pipe = create_pipeline(pipeline_name='transform',
                       data_root='./data/data_for_tfx/',
                       train_pattern = 'train*',
                       val_pattern = 'val*',
                       transform_module_file='./transform.py')
LocalDagRunner().run(pipe)

running bdist_wheel
running build
running build_py
creating build
creating build/lib
copying tf_pipeline.py -> build/lib
copying transform.py -> build/lib
copying train.py -> build/lib
installing to /tmp/tmpinnvfxbd
running install
running install_lib
copying build/lib/transform.py -> /tmp/tmpinnvfxbd
copying build/lib/train.py -> /tmp/tmpinnvfxbd
copying build/lib/tf_pipeline.py -> /tmp/tmpinnvfxbd
running install_egg_info


/home/babych/miniconda3/envs/tfx_14/lib/python3.10/site-packages/setuptools/_distutils/cmd.py:66: SetuptoolsDeprecationWarning: setup.py install is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` directly.
        Instead, use pypa/build, pypa/installer or other
        standards-based tools.

        See https://blog.ganssle.io/articles/2021/10/setup-py-deprecated.html for details.
        ********************************************************************************

!!
  self.initialize_options()


running egg_info
creating tfx_user_code_Transform.egg-info
writing tfx_user_code_Transform.egg-info/PKG-INFO
writing dependency_links to tfx_user_code_Transform.egg-info/dependency_links.txt
writing top-level names to tfx_user_code_Transform.egg-info/top_level.txt
writing manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
reading manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
writing manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
Copying tfx_user_code_Transform.egg-info to /tmp/tmpinnvfxbd/tfx_user_code_Transform-0.0+e89c35a0e5a529113d48973ad2d84d20b6695bc47af49e4340047ec2d90dd258-py3.10.egg-info
running install_scripts
creating /tmp/tmpinnvfxbd/tfx_user_code_Transform-0.0+e89c35a0e5a529113d48973ad2d84d20b6695bc47af49e4340047ec2d90dd258.dist-info/WHEEL
creating '/tmp/tmpxnm4egwc/tfx_user_code_Transform-0.0+e89c35a0e5a529113d48973ad2d84d20b6695bc47af49e4340047ec2d90dd258-py3-none-any.whl' and adding '/tmp/tmpinnvfxbd' to it
adding 'tf_pipeline.py'
a

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Processing ./tfx/pipelines/transform/_wheels/tfx_user_code_Transform-0.0+e89c35a0e5a529113d48973ad2d84d20b6695bc47af49e4340047ec2d90dd258-py3-none-any.whl



[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: pip install --upgrade pip


...

INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:struct2tensor is not available.


INFO:tensorflow:struct2tensor is not available.


INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_decision_forests is not available.


## End of EDA and transformation

After we have transformed data and save them we can go further and develop the model. 